Merging the Data and Storing in a Database/Visualizing Data
Now that you have cleaned and transformed your 3 datasets, you need to load them into a
database. You can choose what kind of database (SQLLite or MySQL, Postgre SQL are all free
options). You will want to load each dataset into SQL Lite as an individual table and then you
must join the datasets together in Python into 1 dataset.
Once all the data is merged together in your database, create 5 visualizations that demonstrate
the data you have cleansed. You should have at least 2 visualizations that have data from more
than one source (meaning, if you have 3 tables, you must have visualizations that span across 2
of the tables – you are also welcome to use your consolidated dataset that you created in the
previous step, if you do that, you have met this requirement).

Submit your code for merging and storing in the database, with your code for the visualizations
along with a 250-500-word summary of what you learned and had to do to complete the
project. You can submit a Jupyter Notebook or a PDF of your code. If you submit a .py file you
need to also include a PDF or attachment of your results.

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math

In [2]:
# reusing the earlier database db_dsc540 for the final project
# Opening a connection object to connect to the database
conn = sqlite3.connect('db_dsc540')

In [3]:
# Reading the data from the table
pd.read_sql_query('SELECT count(*) from FinancialIndicator',conn)

,count(*)
0,4392


In [4]:
pd.read_sql_query('SELECT SECTOR,sum(Revenue) from FinancialIndicator group by Sector',conn)

,Sector,sum(Revenue)
0,Basic Materials,1.083139e+12
1,Communication Services,1.524099e+12
2,Consumer Cyclical,3.178511e+12
3,Consumer Defensive,2.326165e+12
4,Energy,3.219596e+12
5,Financial Services,2.476201e+12
6,Healthcare,2.484020e+12
7,Industrials,2.339055e+12
8,Real Estate,3.007820e+11
9,Technology,2.746504e+12


In [5]:
# Reading the data from the table
pd.read_sql_query('SELECT count(*) from eodstockprice',conn)

,count(*)
0,505


In [10]:
res=pd.read_sql_query('SELECT * from eodstockprice',conn)
res

,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,AAPL,"Oct 16, 2020",2.30,2.32,2.20,2.21,2.21,478100.0
1,AAPL,"Oct 15, 2020",2.28,2.32,2.20,2.31,2.31,766800.0
2,AAPL,"Oct 14, 2020",2.19,2.26,2.18,2.18,2.18,264200.0
3,AAPL,"Oct 13, 2020",2.22,2.24,2.15,2.20,2.20,252100.0
4,AAPL,"Oct 12, 2020",2.24,2.30,2.16,2.19,2.19,452800.0
...,...,...,...,...,...,...,...,...
500,T,"Jun 02, 2020",2.92,2.95,2.79,2.91,2.91,1385500.0
501,T,"Jun 01, 2020",2.95,3.02,2.88,2.88,2.88,873500.0
502,T,"May 29, 2020",2.90,2.97,2.81,2.93,2.93,1055500.0
503,T,"May 28, 2020",3.16,3.21,2.91,2.93,2.93,1402400.0


In [11]:
# these footers should be taken out from the data frame before inserting into the database.
res=pd.read_sql_query('''SELECT * 
                            from eodstockprice
                            where open is null'''
                      ,conn)
res

,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,AAPL,*Close price adjusted for splits.**Adjusted cl...,None,None,None,None,None,None
1,AMZN,*Close price adjusted for splits.**Adjusted cl...,None,None,None,None,None,None
2,LULU,*Close price adjusted for splits.**Adjusted cl...,None,None,None,None,None,None
3,BAC,*Close price adjusted for splits.**Adjusted cl...,None,None,None,None,None,None
4,T,*Close price adjusted for splits.**Adjusted cl...,None,None,None,None,None,None
